# Submission Details

Created dataset for employees with at least 2 tables, and shared the results.

# Tangent 1: 

http://mockaroo.com

## Retrieve data

In [6]:
! curl "https://api.mockaroo.com/api/8c04bca0?count=1000&key=03164ce0" > "Employee Details.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100k    0  100k    0     0  70316      0 --:--:--  0:00:01 --:--:-- 70277


In [7]:
! curl "https://api.mockaroo.com/api/09b05630?count=1000&key=03164ce0" > "Department.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60881    0 60881    0     0  61743      0 --:--:-- --:--:-- --:--:-- 61745


## Setup database

In [8]:
import sqlite3
import pandas as pd
import os

In [9]:
# Define SQL schemas for creating tables
employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(10),
    date_of_birth DATE,
    job_title VARCHAR(100),
    department VARCHAR(50),
    salary DECIMAL(10,2),
    hire_date DATE,
    email VARCHAR(100)
);
"""


departments_schema = """
CREATE TABLE IF NOT EXISTS departments (
    department_id INT PRIMARY KEY,
    department_name VARCHAR(100),
    department_code VARCHAR(50),
    department_head VARCHAR(100),
    department_budget DECIMAL(12,2),
    department_location VARCHAR(100)
);
"""


In [10]:
db_name = 'company.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [11]:
import sqlite3
import pandas as pd
import os

# --- Define SQL Schemas ---
employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(10),
    date_of_birth DATE,
    job_title VARCHAR(100),
    department VARCHAR(50),
    salary DECIMAL(10,2),
    hire_date DATE,
    email VARCHAR(100)
);
"""

departments_schema = """
CREATE TABLE IF NOT EXISTS departments (
    department_id INT PRIMARY KEY,
    department_name VARCHAR(100),
    department_code VARCHAR(50),
    department_head VARCHAR(100),
    department_budget DECIMAL(12,2),
    department_location VARCHAR(100)
);
"""

# --- Define expected column data types ---
COLUMN_DATA_TYPES = {
    'employees': {
        'employee_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'gender': 'object',
        'date_of_birth': 'datetime64[ns]',
        'job_title': 'object',
        'department': 'object',
        'salary': 'float64',
        'hire_date': 'datetime64[ns]',
        'email': 'object'
    },
    'departments': {
        'department_id': 'int64',
        'department_name': 'object',
        'department_code': 'object',
        'department_head': 'object',
        'department_budget': 'float64',
        'department_location': 'object'
    }
}

# --- Database setup ---
db_name = 'company.db'
conn = None  # Initialize connection to None

try:
    # If the database exists, remove it for a clean start
    if os.path.exists(db_name):
        os.remove(db_name)
        print(f"Removed existing database '{db_name}'.")

    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully.")

    # Create tables
    cursor.execute(employees_schema)
    cursor.execute(departments_schema)
    print("Tables 'employees' and 'departments' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/Employee Details.csv': 'employees',
        '/content/Department.csv': 'departments'
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read CSV into DataFrame
            df = pd.read_csv(csv_file)

            # Get expected schema
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # Keep only expected columns
            df = df[df.columns.intersection(expected_cols)]

            # Add missing columns as None (NULL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # Reorder columns to match schema
            df = df[expected_cols]

            # Enforce correct data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")

            # Insert data into SQL table
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f" Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit changes
    conn.commit()
    print("\n Data committed to the database successfully!")

except sqlite3.Error as e:
    print(f" Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f" Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f" Schema definition error: Missing column in COLUMN_DATA_TYPES: {e}")
except Exception as e:
    print(f" Unexpected error: {e}")
finally:
    if conn:
        conn.close()
        print(" Database connection closed.")


Database 'company.db' created and connected successfully.
Tables 'employees' and 'departments' created successfully.

Processing '/content/Employee Details.csv' for table 'employees'...
  -> Data from '/content/Employee Details.csv' loaded into 'employees' table successfully.

Processing '/content/Department.csv' for table 'departments'...
  - Warning: Could not convert column 'department_id' to int64. Error: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'. Leaving as is.
  - Warning: Could not convert column 'department_budget' to float64. Error: could not convert string to float: 'Dollar'. Leaving as is.
  -> Data from '/content/Department.csv' loaded into 'departments' table successfully.

 Data committed to the database successfully!
 Database connection closed.


# Tangent 2: setup API Key using Google's AI Studio

https://aistudio.google.com/


And, the key as `Secrets` in Colab.

### Install Gen AI library

We will be installing of the google-generativeai package, the official Python SDK for the Gemini API.

In [12]:
!pip install google-genai

### Import required modules

In [13]:
from google import genai
from google.colab import userdata

In [14]:
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

# Tangent 3: Prompt Engineering

In [15]:
prompt = """

### ROLE ###
You are an expert Text-to-SQL translator with advanced knowledge of SQL syntax, database schema reasoning, and natural language understanding.
Your job is to generate syntactically correct and semantically accurate SQL queries in SQLite syntax based on user input and the given schema.

### CONTEXT ###
The user is working with a **company management database** containing employee and department data.
Users will provide natural language questions in English, and you must convert them into equivalent SQL SELECT queries that correctly retrieve the requested data.

Here is the database schema:

**Employees Table**
```sql
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(10),
    date_of_birth DATE,
    job_title VARCHAR(100),
    department VARCHAR(50),
    salary DECIMAL(10,2),
    hire_date DATE,
    email VARCHAR(100)
);"""


In [20]:
import json
def get_sql_query_via_gemini(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)

  # Print the raw response text for debugging
  print("Raw model response text:")
  print(response.text)
  print("-" * 20)


  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  # Clean the output to remove potential leading/trailing characters or markdown
  output = response.text.replace('```sqlite\n', '').replace('```', '').strip()

  # Print the cleaned output
  print("Cleaned output from get_sql_query_via_gemini:")
  print(output)
  print("-" * 20)

  return output

In [21]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='company.db'):
    """
    Executes an SQL query on the specified SQLite database
    and returns the results as a pandas DataFrame.
    """

    conn = None
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        print(f"\n Executing query on '{db_name}':\n{query}\n")
        print(f"--- Cleaned query for execution ---\n{query}\n-----------------------------------")


        # Execute the SQL query
        cursor.execute(query)

        # If the query is a SELECT, fetch results
        if query.strip().lower().startswith("select"):
            rows = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            results_df = pd.DataFrame(rows, columns=columns)
            print(" Query executed successfully and data fetched.")
            return results_df
        else:
            # For INSERT, UPDATE, DELETE, CREATE TABLE, etc.
            conn.commit()
            print(" Query executed successfully (no data returned).")
            return None

    except sqlite3.Error as e:
        print(f" Database error: {e}")
        return None

    except Exception as e:
        print(f" Unexpected error: {e}")
        return None

    finally:
        if conn:
            conn.close()

In [22]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query_via_gemini(genai_client, prompt, user_query)
  results = execute_query(output)
  return results

In [23]:
text2sql(genai_client, prompt, " How many employees are there?")

Raw model response text:
```sqlite
SELECT COUNT(*) FROM employees;
```
--------------------
Input Token Count: 256
Thoughts Token Count: 40
Output Token Count: 11
Total Token Count: 307
Cleaned output from get_sql_query_via_gemini:
SELECT COUNT(*) FROM employees;
--------------------

 Executing query on 'company.db':
SELECT COUNT(*) FROM employees;

--- Cleaned query for execution ---
SELECT COUNT(*) FROM employees;
-----------------------------------
 Query executed successfully and data fetched.


,COUNT(*)
0,1000


In [24]:
text2sql(genai_client, prompt, " List the names and job titles of all employees in the 'Sales' department.")

Raw model response text:
```sqlite
SELECT first_name, last_name, job_title
FROM employees
WHERE department = 'Sales';
```
--------------------
Input Token Count: 266
Thoughts Token Count: 45
Output Token Count: 27
Total Token Count: 338
Cleaned output from get_sql_query_via_gemini:
SELECT first_name, last_name, job_title
FROM employees
WHERE department = 'Sales';
--------------------

 Executing query on 'company.db':
SELECT first_name, last_name, job_title
FROM employees
WHERE department = 'Sales';

--- Cleaned query for execution ---
SELECT first_name, last_name, job_title
FROM employees
WHERE department = 'Sales';
-----------------------------------
 Query executed successfully and data fetched.


,first_name,last_name,job_title
0,Dave,Studart,Social Worker
1,Orin,Shury,Internal Auditor
2,Silas,Yitshak,Tax Accountant
3,Elberta,Lensch,Account Coordinator
4,Birch,Chafer,Electrical Engineer
...,...,...,...
197,Concordia,Blissett,Compensation Analyst
198,Smith,Crittal,Operator
199,Ajay,Ledgerton,Account Coordinator
200,Vivien,Server,Professor
